In [35]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import seaborn as sns
import datetime
import plotly.express as px
import plotly


distance = 1

client = Socrata("data.cityofchicago.org", None)
results = client.get('ijzp-q8t2', where = "year > '2019'", limit = 50000)
df1 = pd.DataFrame(data = results)
df1 = df1.rename(columns ={'latitude': 'cust_lat', 'longitude': 'cust_lon'})
df1 = df1.dropna(subset = ['cust_lat'])
df1['date'] = df1['date'].astype(str)
df1['date'] = df1['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f'))
df1['hh:mm'] = df1['date'].dt.strftime('%H:%M')
df1['hour'] = df1['date'].dt.hour.astype(int)
df1['min'] = df1['date'].dt.minute.astype(int)
df1['dt'] = pd.to_datetime(df1['hh:mm'])


df1['event_lat'] = 41.9244051
df1['event_lon'] = -87.6645583

df1[['event_lat', 'event_lon', 'cust_lat', 'cust_lon']] = df1[['event_lat', 'event_lon', 'cust_lat', 'cust_lon']].astype(float) 
df1['lat'] = df1['cust_lat']
df1['lon'] = df1['cust_lon']
df1['event_lon'] = df1['event_lon'] * np.pi / 180
df1['event_lat'] = df1['event_lat'] * np.pi / 180
df1['cust_lon'] = df1['cust_lon'] * np.pi / 180
df1['cust_lat'] = df1['cust_lat'] * np.pi / 180
df1['dlon'] = df1['event_lon'] - df1['cust_lon']
df1['dlat'] = df1['event_lat'] - df1['cust_lat']
df1['a'] = np.sin(df1['dlat']/2.0)**2 + np.sin(df1['dlon']/2.0)**2 * np.cos(df1['cust_lat']) * np.cos(df1['event_lat'])
df1['c'] = 2 * np.arcsin(np.sqrt(df1['a']))
df1['dist'] = (6367 * df1['c']) * 0.621371
df1[f'{distance}+'] = df1[f'dist'] > distance
df1 = df1[df1[f'{distance}+'] == False]



ctype = df1.groupby('primary_type')['id'].nunique().reset_index().sort_values('id', ascending = False).head(10)
dtype = df1.groupby('description')['id'].nunique().reset_index().sort_values('id', ascending = False).head(10)
ltype = df1.groupby('location_description')['id'].nunique().reset_index().sort_values('id', ascending = False).head(10)
ttype = df1.groupby('dt')['id'].nunique().reset_index().sort_values('dt')
df1['date2'] = df1['date'].astype(str)

#plotly.io.orca.config.mapbox_access_token = 'pk.eyJ1Ijoiamlyc2FqYW0iLCJhIjoiY2s3MmhpMmg5MDF5NTNrbTNmZHE3c3pzdCJ9.IA98AyunoPmUFi7ViKnkPg'
#px.set_mapbox_access_token(open("pk.eyJ1Ijoiamlyc2FqYW0iLCJhIjoiY2s3MmhrcDJ2MDI2dzNucGFvNzE4YjNjcyJ9.0dEWMLtQ5qsoGNklI9flAg").read())
fig = px.scatter_mapbox(df1, lat="lat", lon="lon", hover_name = 'primary_type', hover_data = ['description', 'location_description', 'date2']
                        ,color = 'primary_type',zoom = 13, color_continuous_scale=px.colors.cyclical.IceFire, mapbox_style="carto-positron",
                       )
fig.update_layout(hovermode = 'closest')

fig.show()

In [36]:
df2 = df1.groupby('primary_type')['id'].nunique().reset_index()
df2 = df2.sort_values('id', ascending = False).set_index('primary_type')
print(df2)

                         id
primary_type               
THEFT                   196
CRIMINAL DAMAGE          42
DECEPTIVE PRACTICE       42
BATTERY                  35
BURGLARY                 34
ASSAULT                  15
ROBBERY                  13
CRIMINAL TRESPASS        12
OTHER OFFENSE             9
MOTOR VEHICLE THEFT       9
NARCOTICS                 2
WEAPONS VIOLATION         2
PUBLIC PEACE VIOLATION    1
LIQUOR LAW VIOLATION      1
SEX OFFENSE               1
STALKING                  1
KIDNAPPING                1


In [73]:
ttype2 = df1.groupby(['dt', 'primary_type', 'description', 'location_description'])['id'].nunique().reset_index().sort_values('dt')

fig2 = px.bar(ttype2,x = 'dt', y = 'id', color = 'primary_type')
fig2.show()

In [86]:
cat = df1[['primary_type', 'description', 'location_description', 'id']]
cat[['primary_type', 'description', 'location_description']] = cat[['primary_type', 'description', 'location_description']].astype('category')
cat = cat.dropna()

fig3 = px.parallel_categories(df1, dimensions = ['primary_type', 'description', 'location_description']
                             )
fig3.show()
cat.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 412 entries, 83 to 29390
Data columns (total 4 columns):
primary_type            412 non-null category
description             412 non-null category
location_description    412 non-null category
id                      412 non-null object
dtypes: category(3), object(1)
memory usage: 12.9+ KB
